# Семинар 9 - Методы построения оптического потока по последовательности изображений

**Этот семинар содержит оцениваемое домашнее задание**

***

Источник - https://habr.com/ru/post/201406/

$\textbf{Task statement}$: Оптический поток (ОП) – изображение видимого движения, представляющее собой сдвиг каждой точки (пикселя) между двумя изображениями.

По сути, он представляет собой поле скоростей. Суть ОП в том, что для каждой точки изображения $I_{t_0} (\vec{r})$ находится такой вектор сдвига $\delta \vec{r}$, чтобы было соответсвие между исходной точкой и точкой на следущем фрейме $I_{t_1} (\vec{r} + \delta \vec{r})$. В качестве метрики соответвия берут близость интенсивности пикселей, беря во внимание маленькую разницу по времени между кадрами: $\delta{t} = t_{1} - t_{0}$. В более точных методах точку можно привязывать к объекту на основе, например, выделения ключевых точек, а также считать градиенты вокруг точки, лапласианы и проч.

$\textbf{For what}$: Определение собственной скорости, Определение локализации, Улучшение методов трекинга объектов, сегментации, Детектирование событий, Сжатие видеопотока и проч.

![](data/tennis.png)

Разделяют 2 вида оптического потока - плотный (dense) [Farneback method, neural nets], работающий с целым изображением, и выборочный (sparse) [Lucas-Kanade method], работающий с ключевыми точками

In [ ]:
# !wget https://www.bogotobogo.com/python/OpenCV_Python/images/mean_shift_tracking/slow_traffic_small.mp4 -O data/slow_traffic_small.mp4

In [1]:
import cv2
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import IPython

%matplotlib inline

## Lucas-Kanade (sparse)

Пусть $I_{1} = I(x, y, t_{1})$ интенсивность в некоторой точке (x, y) на первом изображении (т. е. в момент времени t). На втором изображении эта точка сдвинулась на (dx, dy), при этом прошло время dt, тогда $I_{2} = I(x + dx, y + dx, t_{1} + dt) \approx I_{1} + I_{x}dx + I_{y}dy +  I_{t}dt$. Из постановки задачи следует, что интенсивность пикселя не изменилась, тогда $I_{1} = I_{2}$. Далее определяем $dx, dy$.

Самое простое решение проблемы – алгоритм Лукаса-Канаде. У нас же на изображении объекты размером больше 1 пикселя, значит, скорее всего, в окрестности текущей точки у других точек будут примерно такие же сдвиги. Поэтому мы возьмем окно вокруг этой точки и минимизируем (по МНК) в нем суммарную погрешность с весовыми коэффициентами, распределенными по Гауссу, то есть так, чтобы наибольший вес имели пиксели, ближе всего находящиеся к исследуемому.

**Полезные материалы:** 
- цикл видео-лекций от First Principles of Computer Vision, посвященный Optical Flow и алгоритму Lucas-Kanade: https://youtube.com/playlist?list=PL2zRqk16wsdoYzrWStffqBAoUY8XdvatV

### Вопрос 1

Перечислите три основных предположения, на которых базируется метод Lucas-Kanade. Почему каждое из них важно для корректной работы алгоритма?

**Ответ:**

Яркостная постоянство: интенсивность каждого пикселя остаётся неизменной при смещении между кадрами, то есть I(x,y,t)=I(x+u,y+v,t+1). Это важно, чтобы линейная аппроксимация оставалась корректной и позволяла связать пространственные и временные градиенты яркости 

Малые перемещения: смещения u,v между соседними кадрами должны быть малы (обычно < 1 пикселя), чтобы первые члены разложения Тейлора давали достаточно точную аппроксимацию нелинейного сдвига 

Пространственная когерентность (гладкость потока): в небольшом окне вокруг точки поток считается постоянным, что даёт дополнительные уравнения для надёжного решения СЛАУ и уменьшает чувствительность к шуму


### Вопрос 2

Объясните, зачем нужен пирамидальный подход в алгоритме Lucas-Kanade. Какую проблему он решает и как именно?

**Ответ:**
При больших смещениях между кадрами разложение Тейлора теряет точность, так как нарушается предположение малых перемещений. Пирамидальная схема (снижение разрешения на верхнем уровне и постепенный переход к исходному) преобразует крупные смещения в малые, где алгоритм Lucas–Kanade быстро оценивает грубый поток

### Вопрос 3

С какими проблемами может столкнуться алгоритм Lucas-Kanade при отслеживании точек на видео? Назовите минимум три ограничения.

**Ответ:**

Окклюзии: скрытие или появление объектов приводит к невозможности соотнести пиксели между кадрами и вызывает ошибки в оценке потока 

Изменения освещённости: нарушение яркостного постоянства (например, при мерцании или затенении) искажает связь яркости между кадрами 

Текстурно-плохие области: в однотонных или слабоструктурированных зонах нет устойчивых градиентов, что делает матрицу нормальных уравнений плохо обусловленной

### Задание 1

Напишите реализацию Лукаса-Канаде c помощью numpy и cv2. Сравните с реализацией `cv2.calcOpticalFlowPyrLK`.

In [10]:
import cv2
import numpy as np


def build_image_pyramid(image, num_levels, scale_factor=0.5):
    pyramid = [image.copy()]
    for i in range(1, num_levels):
        prev = pyramid[-1]
        h, w = prev.shape[:2]
        new_size = (max(1, int(w * scale_factor)), max(1, int(h * scale_factor)))
        resized = cv2.resize(prev, new_size, interpolation=cv2.INTER_LINEAR)
        pyramid.append(resized)
    return pyramid


def compute_image_gradients(image):
    Ix = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    Iy = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    return Ix, Iy


def compute_lk_optical_flow_point(Ix, Iy, It, window_size=5, eig_threshold=1e-4):
    half = window_size // 2
    Ix_w = Ix.flatten()
    Iy_w = Iy.flatten()
    It_w = It.flatten()
    A11 = np.sum(Ix_w * Ix_w)
    A12 = np.sum(Ix_w * Iy_w)
    A22 = np.sum(Iy_w * Iy_w)
    G = np.array([[A11, A12], [A12, A22]])
    # Проверка обусловленности через собственные значения
    eigs = np.linalg.eigvals(G)
    if np.min(eigs) < eig_threshold:
        return None, None
    b = -np.array([np.sum(Ix_w * It_w), np.sum(Iy_w * It_w)])
    try:
        nu = np.linalg.inv(G) @ b
        return nu[0], nu[1]
    except np.linalg.LinAlgError:
        return None, None


def compute_lk_optical_flow_for_patch(prev_patch, curr_patch, window_size=5):
    Ix, Iy = compute_image_gradients(prev_patch)
    It = (curr_patch - prev_patch).astype(np.float64)
    return compute_lk_optical_flow_point(Ix, Iy, It, window_size)


def track_point_with_pyramid_lk(prev_pyr, curr_pyr, point,
                               window_size=15, max_iterations=10, epsilon=0.01):
    levels = len(prev_pyr)
    # Изначальная точка на уровне 0
    u, v = 0.0, 0.0
    # Координаты в текущих уровнях
    x, y = point
    # Начинаем с верхнего (коэффициент scale_factor**(levels-1))
    for lvl in reversed(range(levels)):
        scale = 1.0 / (2 ** lvl)
        x_lvl = x * scale + u
        y_lvl = y * scale + v
        prev = prev_pyr[lvl]
        curr = curr_pyr[lvl]
        h, w = prev.shape
        # Итеративное уточнение
        for _ in range(max_iterations):
            x0, y0 = int(round(x_lvl)), int(round(y_lvl))
            half = window_size // 2
            # Проверка границ
            if x0 - half < 0 or x0 + half >= w or y0 - half < 0 or y0 + half >= h:
                return None
            prev_patch = prev[y0-half:y0+half+1, x0-half:x0+half+1]
            curr_patch = curr[y0-half:y0+half+1, x0-half:x0+half+1]
            du, dv = compute_lk_optical_flow_for_patch(prev_patch, curr_patch, window_size)
            if du is None:
                return None
            x_lvl += du
            y_lvl += dv
            if np.hypot(du, dv) < epsilon:
                break
        # Переносим смещение на следующий (более высокое разрешение)
        u = x_lvl - x * scale
        v = y_lvl - y * scale
        # Увеличиваем базовую точку для следующего уровня
        u *= 2
        v *= 2
    # Финальная позиция на уровне 0
    new_x = x + u / 2
    new_y = y + v / 2
    return (new_x, new_y)


def lucas_kanade_optical_flow(prev_frame, curr_frame, points,
                              window_size=15, num_pyramid_levels=3,
                              max_iterations=10, epsilon=0.01):
    # Приводим к grayscale и нормализуем
    if prev_frame.ndim == 3:
        prev = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY).astype(np.float64) / 255.0
        curr = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY).astype(np.float64) / 255.0
    else:
        prev = prev_frame.astype(np.float64) / 255.0
        curr = curr_frame.astype(np.float64) / 255.0

    prev_pyr = build_image_pyramid(prev, num_pyramid_levels)
    curr_pyr = build_image_pyramid(curr, num_pyramid_levels)

    new_points = []
    status = []
    for pt in points:
        res = track_point_with_pyramid_lk(
            prev_pyr, curr_pyr, pt, window_size, max_iterations, epsilon
        )
        if res is None:
            new_points.append(pt)
            status.append(0)
        else:
            new_points.append(res)
            status.append(1)
    return np.array(new_points, dtype=np.float32), np.array(status, dtype=np.uint8)


def demo_optical_flow(video_path='data/slow_traffic_small.mp4', output_path='output_my_LK.mp4'):
    """
    Демонстрация работы алгоритма на видео.

    Args:
        video_path: Путь к входному видео
        output_path: Путь для сохранения результата
    """
    # Открываем видео
    cap = cv2.VideoCapture(video_path)

    # Получаем параметры видео
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Настраиваем запись выходного видео
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    # Параметры для обнаружения углов Shi-Tomasi
    feature_params = dict(
        maxCorners=100,
        qualityLevel=0.3,
        minDistance=7,
        blockSize=7
    )

    # Берем первый кадр и находим в нем углы
    ret, old_frame = cap.read()
    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
    p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)
    p0 = p0.reshape(-1, 2)  # Преобразуем в формат [[x1, y1], [x2, y2], ...]

    # Сохраняем изначальные точки для отслеживания через все видео
    initial_points = p0.copy()

    # Создаем маску для рисования
    mask = np.zeros_like(old_frame)

    # Создаем случайные цвета для визуализации
    color = np.random.randint(0, 255, (len(p0), 3))

    from tqdm import tqdm
    for i in tqdm(range(length - 1)):  # -1 потому что первый кадр мы уже прочитали
        ret, frame = cap.read()

        if not ret:
            print('No frames grabbed!')
            break

        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Вычисляем оптический поток с помощью нашей реализации
        p1, st = lucas_kanade_optical_flow(
            old_gray,
            frame_gray,
            p0,
            window_size=15,
            num_pyramid_levels=3
        )

        # Выбираем хорошие точки
        good_new = p1[st == 1]
        good_old = p0[st == 1]

        # Рисуем треки
        for i, (new, old) in enumerate(zip(good_new, good_old)):
            a, b = new
            c, d = old
            mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), color[i % len(color)].tolist(), 2)
            frame = cv2.circle(frame, (int(a), int(b)), 5, color[i % len(color)].tolist(), -1)

        # Объединяем кадр и маску
        img = cv2.add(frame, mask)

        # Записываем результат
        out.write(img)

        # Обновляем предыдущий кадр
        old_gray = frame_gray.copy()

        # Обновляем точки, но только те, которые успешно отслежены
        p0[st == 1] = good_new

    # Освобождаем ресурсы
    cap.release()
    out.release()

    print(f"Результат сохранен в {output_path}")
    return output_path

In [11]:
result_path = demo_optical_flow(video_path='data/slow_traffic_small.mp4', output_path='output_my_LK.mp4')

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
  0%|          | 0/913 [00:00<?, ?it/s]

100%|██████████| 913/913 [00:58<00:00, 15.66it/s]

Результат сохранен в output_my_LK.mp4


### Релизация OpenCV - cv2.calcOpticalFlowPyrLK

In [6]:
def demo_optical_flow_opencv(video_path='data/slow_traffic_small.mp4', output_path='output_my_LK.mp4'):
    """
    Демонстрация работы алгоритма на видео с использованием cv2.calcOpticalFlowPyrLK.

    Args:
        video_path: Путь к входному видео
        output_path: Путь для сохранения результата
    """
    # Открываем видео
    cap = cv2.VideoCapture(video_path)

    # Получаем параметры видео
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Настраиваем запись выходного видео
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    # Параметры для обнаружения углов Shi-Tomasi
    feature_params = dict(
        maxCorners=100,
        qualityLevel=0.3,
        minDistance=7,
        blockSize=7
    )

    # Параметры для Lucas-Kanade оптического потока
    lk_params = dict(
        winSize=(15, 15),
        maxLevel=3,
        criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03)
    )

    # Берем первый кадр и находим в нем углы
    ret, old_frame = cap.read()
    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
    p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)

    # Создаем маску для рисования
    mask = np.zeros_like(old_frame)

    # Создаем случайные цвета для визуализации
    color = np.random.randint(0, 255, (100, 3))

    from tqdm import tqdm
    for i in tqdm(range(length - 1)):  # -1 потому что первый кадр мы уже прочитали
        ret, frame = cap.read()

        if not ret:
            print('No frames grabbed!')
            break

        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Вычисляем оптический поток с помощью встроенной функции cv2.calcOpticalFlowPyrLK
        p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

        # Выбираем хорошие точки
        if p1 is not None:
            good_new = p1[st == 1]
            good_old = p0[st == 1]

        # Рисуем треки
        for i, (new, old) in enumerate(zip(good_new, good_old)):
            a, b = new.ravel()
            c, d = old.ravel()
            mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), color[i % len(color)].tolist(), 2)
            frame = cv2.circle(frame, (int(a), int(b)), 5, color[i % len(color)].tolist(), -1)

        # Объединяем кадр и маску
        img = cv2.add(frame, mask)

        # Записываем результат
        out.write(img)

        # Обновляем предыдущий кадр
        old_gray = frame_gray.copy()

        # Обновляем точки, но только те, которые успешно отслежены
        p0 = good_new.reshape(-1, 1, 2)

    # Освобождаем ресурсы
    cap.release()
    out.release()

    print(f"Результат сохранен в {output_path}")
    return output_path

In [7]:
result_path = demo_optical_flow_opencv(video_path='data/slow_traffic_small.mp4', output_path='output_opencv_LK.mp4')

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
100%|██████████| 913/913 [00:08<00:00, 105.75it/s]

Результат сохранен в output_opencv_LK.mp4


### Задание 2

В базовой реализации у кода есть одна важная проблема - ключевые точки инициализируются единожды. В реальных задачах необходимо отслеживать точки, которые исчезают из кадра и появляются в других местах. Реализуйте механизм, который будет отслеживать точки, которые пропадают из кадра и добавлять новые точки в те места, где они появляются. Для этого вам нужно будет реализовать механизм поиска новых точек на изображении.

In [ ]:
video_path='data/slow_traffic_small.mp4'
cap = cv2.VideoCapture(video_path)
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = cap.get(cv2.CAP_PROP_FPS)

fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter('output_fixed_LK.mp4', fourcc, fps, (width, height))

feature_params = dict(
    maxCorners = 100,
    qualityLevel = 0.3,
    minDistance = 7,
    blockSize = 7,
)
lk_params = dict(
    winSize  = (15, 15),
    maxLevel = 2,
    criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03),
)
color = np.random.randint(0, 255, (100, 3))
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
mask = np.zeros_like(old_frame)

for i in tqdm(range(length)):
    ret, frame = cap.read()
    if not ret:
        print('No frames grabbed!')
        break

    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    p1, st, _ = cv2.calcOpticalFlowPyrLK(
        prevImg=old_gray,
        nextImg=frame_gray,
        prevPts=p0,
        nextPts=None,
        **lk_params,
    )

    if p1 is not None:
        good_new = p1[st==1]
        good_old = p0[st==1]

    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
        frame = cv2.circle(frame, (int(a), int(b)), 5, color[i].tolist(), -1)
    img = cv2.add(frame, mask)

    if p1 is not None and len(good_new) > 0:
        old_gray = frame_gray.copy()
        p0 = good_new.reshape(-1, 1, 2)

    # re-detect features periodically
    if len(p0) < feature_params['maxCorners'] // 2 and \
        any([(p > old_frame.shape).all() for p in p0.ravel()]):
        p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)

    out.write(img)

cap.release()
out.release()

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
  1%|          | 9/914 [00:00<00:10, 85.26it/s]

100%|█████████▉| 913/914 [00:10<00:00, 86.23it/s] 


No frames grabbed!


### Вопрос 4

В чем основное отличие разреженного (sparse) оптического потока Lucas-Kanade от плотного (dense) оптического потока (например, метода Farneback)?

**Ответ:**
Lucas–Kanade (sparse): вычисляет поток только в заранее выбранных точках-характеристиках (углы Shi-Tomasi, FAST и т.д.).

Farneback (dense): аппроксимирует поле движения для каждого пикселя кадра, давая непрерывную карту смещений.

## Farneback (dense)

Метод Farneback носит несколько более глобальный характер, чем метод Лукаса-Канаде. Он опирается на предположение о том, что на всем изображении оптический поток будет достаточно гладким.

# Вопрос 5

Перечислите основные шаги алгоритма Farneback для расчета оптического потока.

**Ответ:**

1.Представление локального окна как квадратичной полиномиальной поверхности (двумерный полином второго порядка).   
2. Оенка сдвига между двумя такими поверхностями путём решения линейной системы.   
3. Итеративное уточнение: свернуть, сместить изображение, пересчитать.  


### Вопрос 6

Каким образом в методе Farneback обрабатываются большие смещения объектов между кадрами?

**Ответ:**
Используется многоуровневая (coarse-to-fine) пирамида: сначала поток оценивается на сильно сжатых кадрах, где даже большой реальный сдвиг выглядит маленьким, затем результат постепенно интерполируется и уточняется на более высоких разрешениях

In [18]:
def demo_optical_flow_farneback_opencv(video_path='data/slow_traffic_small.mp4', output_path='output_Farneback.mp4'):
    """
    Демонстрация работы алгоритма плотного оптического потока Farneback на видео.

    Args:
        video_path: Путь к входному видео
        output_path: Путь для сохранения результата
    """
    # Открываем видео
    cap = cv2.VideoCapture(video_path)

    # Получаем параметры видео
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Настраиваем запись выходного видео
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    # Берем первый кадр и преобразуем его в оттенки серого
    ret, frame1 = cap.read()
    if not ret:
        print('Не удалось прочитать видео')
        return None

    prvs = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

    # Создаем HSV-изображение для визуализации потока
    hsv = np.zeros_like(frame1)
    hsv[..., 1] = 255  # Насыщенность устанавливаем на максимум

    from tqdm import tqdm
    for i in tqdm(range(length - 1)):  # -1 потому что первый кадр мы уже прочитали
        ret, frame2 = cap.read()

        if not ret:
            print('No frames grabbed!')
            break

        next_frame = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

        # Вычисляем оптический поток методом Farneback
        # Параметры:
        # - 0.5: коэффициент масштабирования для пирамиды изображений
        # - 3: кол-во уровней пирамиды
        # - 15: размер окна для усреднения
        # - 3: число итераций на каждом уровне пирамиды
        # - 5: размер окна для полиномиальной аппроксимации
        # - 1.2: стандартное отклонение для сглаживания
        flow = cv2.calcOpticalFlowFarneback(
            prvs, next_frame, None,
            0.5, 3, 15, 3, 5, 1.2, 0
        )

        # Преобразуем векторы потока из декартовых координат в полярные
        mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])

        # Кодируем направление потока как оттенок (hue)
        hsv[..., 0] = ang * 180 / np.pi / 2

        # Кодируем величину потока как яркость (value)
        hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)

        # Преобразуем HSV в BGR для отображения
        bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

        # Записываем результат
        out.write(bgr)

        # Обновляем предыдущий кадр
        prvs = next_frame

    # Освобождаем ресурсы
    cap.release()
    out.release()

    print(f"Результат сохранен в {output_path}")
    return output_path

In [19]:
result_path = demo_optical_flow_farneback_opencv(video_path='data/slow_traffic_small.mp4', output_path='output_opencv_farneback.mp4')

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
100%|██████████| 913/913 [02:07<00:00,  7.18it/s]

Результат сохранен в output_opencv_farneback.mp4


### Вопрос 7

Как влияет предварительная обработка изображений (фильтрация шума, выравнивание гистограмм) на качество оптического потока, получаемого методом Farneback? Предложите оптимальный пайплайн предобработки.

**Ответ:**
1. Convert → grayscale (цветовая информация не используется).     
2. Gaussian blur σ≈1–1.5 px — убираем высокочастотный шум.    
3. CLAHE (clip ≈ 2.0, tile ≈ 8×8) — лёгкое локальное выравнивание контраста.    
4. I′ = I / mean(I) или линейное выравнивание среднего/STD между парой кадров.      
5. Global motion compensation (feature-based homography) для видео с дрожанием.    
6. Farneback (пирамида = 5–6 уровней, winsize ≈ 9–15, iterations ≈ 3).     
